In [1]:
# Notebook for visualising breakthrough infections in a different way
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget  
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


# Load data

In [2]:
dfPop = pd.read_excel('DKfolketal2021clean.xlsx')
dfPop.head()


,Alder,Antal
0,0,61191
1,1,61663
2,2,61967
3,3,62185
4,4,62844


In [3]:
ssidatapath = "ssi_vacc"
rootdir = os.getcwd() +"\\" + ssidatapath



for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])
    
print(latestdir)
print(latestDate)

c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-10-12
2021-10-12 00:00:00


In [4]:

filename = '/Vaccine_DB/Vaccinationer_region_aldgrp_koen.csv'
curPath = latestdir+ filename

df_vacc = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')
df_vacc[df_vacc['Aldersgruppe'] == '20-29']['Antal faerdigvacc.'].astype(int).sum()

579846

In [5]:
# df_vacc

In [6]:
# We have to collect from previous dates

curAgeGroup = '20-29'
curPopSize = dfPop[20:29].Antal.sum()

agedf = pd.DataFrame()

allDates = []
allVacc = []

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])
    
    try:
        filename = '/Vaccine_DB/Vaccinationer_region_aldgrp_koen.csv'
        curPath = latestdir+ filename
        df_vacc = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')
        curVacc = df_vacc[df_vacc['Aldersgruppe'] == '20-29']['Antal faerdigvacc.'].astype(int).sum()
        
        allDates.append(latestDate)
        allVacc.append(curVacc)
    except:
        2+2
        
allDates = np.array(allDates)    
allVacc = np.array(allVacc)

agedf['Dates'] = allDates
agedf['Vaccinated'] = allVacc
agedf['Unvaccinated'] = curPopSize - allVacc
agedf.tail()

,Dates,Vaccinated,Unvaccinated
82,2021-10-06,577681,122785
83,2021-10-07,578164,122302
84,2021-10-08,578474,121992
85,2021-10-11,579553,120913
86,2021-10-12,579846,120620


In [7]:
# df_vacc
# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.fill_between(allDates,np.ones(allVacc.shape)*curPopSize,color='xkcd:light red')
ax1.fill_between(allDates,allVacc,color='xkcd:blue')
ax2.fill_between(allDates,100*np.ones(allVacc.shape),color='xkcd:light red')
ax2.fill_between(allDates,100*allVacc/curPopSize,color='xkcd:blue')

ax1.set_ylim(bottom=0,top=curPopSize)
ax2.set_ylim(bottom=0,top=100)

ax1.set_xlim([allDates.min(),allDates.max()])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
agedf.tail(20)

,Dates,Vaccinated,Unvaccinated
67,2021-09-17,564552,135914
68,2021-09-18,565584,134882
69,2021-09-19,566396,134070
70,2021-09-20,567024,133442
71,2021-09-21,568443,132023
72,2021-09-22,569452,131014
73,2021-09-23,570232,130234
74,2021-09-24,570962,129504
75,2021-09-27,572883,127583
76,2021-09-28,573613,126853


In [9]:
# Define datapath and load
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"\\" + ssidatapath


# Data is (only) in the file from the most recent tuesday. 
# Should be made smarter, but here hardcoded
tuePath = 'overvaagningsdata-covid19-14092021-jrhb'
# tuePath = 'SSI_data_2021-09-14'
# tuePath = 'SSI_data_2021-09-28'
# tuePath = 'SSI_data_2021-10-05'
tuePath = 'SSI_data_2021-10-12'

dirPath = rootdir + '\\' + tuePath + '\\'

df1 = pd.read_csv(dirPath+'gennembrudsinfektioner_table1.csv',delimiter=';')
df2_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_cases.csv',delimiter=';')
df2_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_indlagte.csv',delimiter=';')
df2_R = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_repositive.csv',delimiter=';')
df3 = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_alle.csv',delimiter=';')
df3_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_cases.csv',delimiter=';')
df3_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_indlagte.csv',delimiter=';')

In [10]:
# Flip dataframe to be like df1
casedf = pd.DataFrame()
casedf_once = pd.DataFrame()
casedf_both = pd.DataFrame()
casedf_full = pd.DataFrame()
casedf_none = pd.DataFrame()

weekNames = df1.Ugenummer
casedf['Ugenummer'] = weekNames
casedf_none['Ugenummer'] = weekNames
casedf_once['Ugenummer'] = weekNames
casedf_both['Ugenummer'] = weekNames
casedf_full['Ugenummer'] = weekNames

for k in range(0,len(df2_C)):
    curRow = df2_C.loc[k]
    curAge = df2_C.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        curFull.append(thisFull)
    
    # Add to dataframe
    casedf[curAge+'_Ingen vaccination'] = curNone
    casedf[curAge+'_Første vaccination'] = curOnce
    casedf[curAge+'_Anden vaccination'] = curBoth 
    casedf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    casedf_none[curAge] = curNone
    casedf_once[curAge] = curOnce
    casedf_both[curAge] = curBoth 
    casedf_full[curAge] = curFull
    

# df2_C
# curRow
casedf
# cur

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 40,222,10,17,17,75,4,70,70,306,...,176,176,2,3,78,78,1143,90,1690,1679
1,Uge 39,162,13,16,14,76,4,28,28,231,...,119,119,4,2,50,50,932,90,1173,1163
2,Uge 38,171,14,8,6,57,7,33,33,209,...,128,128,3,0,75,75,812,77,964,951
3,Uge 37,148,12,4,3,61,3,14,12,209,...,83,83,2,0,48,48,753,74,781,769
4,Uge 36,183,29,16,8,111,5,38,34,311,...,116,116,3,0,67,67,1130,173,1101,1066
5,Uge 35,330,75,25,15,131,24,53,52,404,...,115,115,11,2,68,67,1580,271,1475,1407
6,Uge 34,493,126,37,10,221,33,71,67,602,...,148,148,9,1,64,64,2310,471,1698,1564
7,Uge 33,590,155,30,1,290,41,99,83,798,...,109,109,3,0,63,63,2750,664,1662,1408
8,Uge 32,483,173,17,1,336,73,122,85,1055,...,101,100,7,2,46,46,2926,1076,1570,1210
9,Uge 31,444,102,6,0,341,138,130,64,991,...,70,70,0,0,36,36,2775,1353,1324,992


In [11]:
casedf_none
# Uge 35 startede 30/08-2021

,Ugenummer,12-15,16-19,20-29,30-39,40-49,50-59,60-69,70-79,80+,12+
0,Uge 40,222,75,306,283,157,65,28,5,2,1143
1,Uge 39,162,76,231,216,132,83,22,6,4,932
2,Uge 38,171,57,209,183,109,53,20,7,3,812
3,Uge 37,148,61,209,173,90,49,15,6,2,753
4,Uge 36,183,111,311,285,128,75,30,4,3,1130
5,Uge 35,330,131,404,373,202,95,26,8,11,1580
6,Uge 34,493,221,602,496,303,136,38,12,9,2310
7,Uge 33,590,290,798,607,287,129,34,12,3,2750
8,Uge 32,483,336,1055,588,287,131,35,4,7,2926
9,Uge 31,444,341,991,561,283,129,21,5,0,2775


In [12]:
# Flip dataframe to be like df1
hospdf = pd.DataFrame()
hospdf_once = pd.DataFrame()
hospdf_both = pd.DataFrame()
hospdf_full = pd.DataFrame()
hospdf_none = pd.DataFrame()

weekNames = df1.Ugenummer
hospdf['Ugenummer'] = weekNames
hospdf_none['Ugenummer'] = weekNames
hospdf_once['Ugenummer'] = weekNames
hospdf_both['Ugenummer'] = weekNames
hospdf_full['Ugenummer'] = weekNames

for k in range(0,len(df2_H)):
    curRow = df2_H.loc[k]
    curAge = df2_H.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        curFull.append(thisFull)
    
    # Add to dataframe
    hospdf[curAge+'_Ingen vaccination'] = curNone
    hospdf[curAge+'_Første vaccination'] = curOnce
    hospdf[curAge+'_Anden vaccination'] = curBoth 
    hospdf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    hospdf_none[curAge] = curNone
    hospdf_once[curAge] = curOnce
    hospdf_both[curAge] = curBoth 
    hospdf_full[curAge] = curFull
    

# df2_H
# curRow
hospdf
# cur

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 40,2,0,0,0,2,0,1,1,10,...,10,10,1,1,22,22,49,2,53,53
1,Uge 39,0,0,1,1,0,1,0,0,9,...,15,15,1,1,11,11,37,4,42,42
2,Uge 38,0,0,0,0,1,0,0,0,3,...,17,17,2,0,24,24,30,2,54,54
3,Uge 37,0,0,0,0,0,0,0,0,4,...,11,11,0,0,18,18,34,5,44,44
4,Uge 36,1,0,0,0,0,0,0,0,19,...,14,14,3,0,20,20,79,9,59,58
5,Uge 35,4,0,0,0,3,0,0,0,18,...,12,12,9,2,18,17,120,13,48,46
6,Uge 34,3,1,0,0,5,0,1,1,18,...,22,22,6,0,17,17,121,11,59,59
7,Uge 33,2,0,0,0,6,0,1,0,35,...,16,16,3,1,11,11,136,13,58,56
8,Uge 32,4,1,0,0,3,1,1,1,25,...,10,10,4,0,9,9,107,29,37,34
9,Uge 31,2,0,0,0,2,1,0,0,21,...,6,6,0,0,13,13,65,14,36,34


In [13]:

# import datetime
# datetime.date(2021, 8, 30).isocalendar()[1]

iniIndex = 0
endIndex = 1
numCaseUnvacc = hospdf_none.loc[iniIndex:endIndex,'20-29'].sum()
numCaseVacc = hospdf_full.loc[iniIndex:endIndex,'20-29'].sum()

iniDate = np.datetime64('2021-08-30')
endDate = iniDate + np.timedelta64(13,'D')

period_agedf = agedf[(agedf.Dates >= iniDate) & (agedf.Dates <= endDate)]

meanUnvacc = period_agedf['Unvaccinated'].mean()
meanVacc = period_agedf['Vaccinated'].mean()

print(f'Mellem {iniDate} og {endDate}, blev der:')
print(f'Indlagt {numCaseVacc} vaccinerede personer mellem 20 og 29, ud af {meanVacc:.0f} vaccinerede')
print(f'Indlagt {numCaseUnvacc} uvaccinerede personer mellem 20 og 29, ud af {meanUnvacc:.0f} uvaccinerede')
# print(f'- Ud af {meanVacc:.0f} vacccinerede personer, indlagt {numCaseVacc} personer')
# print(f'- Ud af {meanUnvacc:.0f} uvacccinerede personer, indlagt {numCaseUnvacc} personer')



Mellem 2021-08-30 og 2021-09-12, blev der:
Indlagt 5 vaccinerede personer mellem 20 og 29, ud af 548466 vaccinerede
Indlagt 19 uvaccinerede personer mellem 20 og 29, ud af 152000 uvaccinerede


In [14]:
df2_H
hospdf_none

,Ugenummer,12-15,16-19,20-29,30-39,40-49,50-59,60-69,70-79,80+,12+
0,Uge 40,2,2,10,12,9,7,3,3,1,49
1,Uge 39,0,0,9,5,6,7,5,4,1,37
2,Uge 38,0,1,3,7,4,5,4,4,2,30
3,Uge 37,0,0,4,9,6,11,4,0,0,34
4,Uge 36,1,0,19,15,21,9,8,3,3,79
5,Uge 35,4,3,18,31,23,16,11,5,9,120
6,Uge 34,3,5,18,40,19,10,10,10,6,121
7,Uge 33,2,6,35,32,27,18,9,4,3,136
8,Uge 32,4,3,25,30,21,10,9,1,4,107
9,Uge 31,2,2,21,13,11,10,3,3,0,65


In [15]:
# Flip dataframe to be like df1
incidf = pd.DataFrame()
incidf_once = pd.DataFrame()
incidf_both = pd.DataFrame()
incidf_full = pd.DataFrame()
incidf_none = pd.DataFrame()

weekNames = df1.Ugenummer
incidf['Ugenummer'] = weekNames
incidf_none['Ugenummer'] = weekNames
incidf_once['Ugenummer'] = weekNames
incidf_both['Ugenummer'] = weekNames
incidf_full['Ugenummer'] = weekNames

for k in range(0,len(df3_C)):
    curRow = df3_C.loc[k]
    curAge = df3_C.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        thisNone = pd.to_numeric(thisNone.replace(',','.'))
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        thisOnce = pd.to_numeric(thisOnce.replace(',','.'))
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        thisBoth = pd.to_numeric(thisBoth.replace(',','.'))
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        thisFull = pd.to_numeric(thisFull.replace(',','.'))
        curFull.append(thisFull)
    
    # Add to dataframe
    incidf[curAge+'_Ingen vaccination'] = curNone
    incidf[curAge+'_Første vaccination'] = curOnce
    incidf[curAge+'_Anden vaccination'] = curBoth 
    incidf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    incidf_none[curAge] = curNone
    incidf_once[curAge] = curOnce
    incidf_both[curAge] = curBoth 
    incidf_full[curAge] = curFull
    

# df2_H
# curRow
incidf
# cur

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 40,319.2,86.2,10.0,10.3,231.1,88.0,33.0,33.1,196.3,...,32.3,32.3,22.3,346.8,29.6,29.7,198.4,134.2,40.6,40.5
1,Uge 39,227.8,89.2,9.7,8.8,231.1,78.3,13.2,13.3,146.4,...,21.8,21.8,44.1,221.7,19.0,19.0,159.8,117.2,28.3,28.2
2,Uge 38,233.4,70.9,5.0,4.0,170.2,118.4,15.7,15.8,130.6,...,23.4,23.5,32.6,0.0,28.4,28.5,137.0,82.4,23.3,23.2
3,Uge 37,194.8,44.1,2.7,2.2,178.3,43.3,6.7,5.8,128.2,...,15.2,15.2,21.4,0.0,18.2,18.2,124.6,64.4,19.0,19.0
4,Uge 36,229.7,75.4,12.0,7.4,314.9,60.4,18.4,16.7,186.5,...,21.2,21.2,31.8,0.0,25.4,25.4,182.5,117.3,27.1,27.0
5,Uge 35,387.9,126.5,23.2,20.3,357.4,240.5,25.9,26.0,235.0,...,21.1,21.1,115.4,179.9,25.7,25.3,247.1,130.0,37.0,37.0
6,Uge 34,524.6,149.8,49.8,25.9,577.0,257.4,35.5,34.8,336.9,...,27.1,27.1,93.0,87.3,24.2,24.2,345.5,151.1,44.0,43.8
7,Uge 33,561.9,141.8,77.3,9.9,718.7,232.8,51.4,46.2,426.2,...,20.0,20.0,30.7,0.0,23.7,23.8,390.2,139.2,45.3,42.6
8,Uge 32,398.0,141.3,164.0,44.1,793.4,259.0,67.7,54.0,534.7,...,18.5,18.4,70.7,135.3,17.3,17.3,389.8,142.5,46.9,39.3
9,Uge 31,292.5,101.2,255.4,0.0,765.4,285.9,82.5,52.0,470.9,...,12.8,12.9,0.0,0.0,13.5,13.6,337.8,143.4,42.8,34.9


In [16]:
# Flip dataframe to be like df1
inciHospdf = pd.DataFrame()
inciHospdf_once = pd.DataFrame()
inciHospdf_both = pd.DataFrame()
inciHospdf_full = pd.DataFrame()
inciHospdf_none = pd.DataFrame()

weekNames = df1.Ugenummer
inciHospdf['Ugenummer'] = weekNames
inciHospdf_none['Ugenummer'] = weekNames
inciHospdf_once['Ugenummer'] = weekNames
inciHospdf_both['Ugenummer'] = weekNames
inciHospdf_full['Ugenummer'] = weekNames

for k in range(0,len(df3_H)):
    curRow = df3_H.loc[k]
    curAge = df3_H.loc[k,'Aldersgruppe']
    
    curNone = []
    curOnce = []
    curBoth = []
    curFull = []
    for curW in weekNames:
        # print(curW)
        
        thisNone = curRow[curW+'_Ingen vaccination']
        thisNone = pd.to_numeric(thisNone.replace(',','.'))
        curNone.append(thisNone)
        
        thisOnce = curRow[curW+'_Første vaccination']
        thisOnce = pd.to_numeric(thisOnce.replace(',','.'))
        curOnce.append(thisOnce)
        
        thisBoth = curRow[curW+'_Anden vaccination']
        thisBoth = pd.to_numeric(thisBoth.replace(',','.'))
        curBoth.append(thisBoth)
        
        thisFull = curRow[curW+'_Fuld vaccineeffekt']
        thisFull = pd.to_numeric(thisFull.replace(',','.'))
        curFull.append(thisFull)
    
    # Add to dataframe
    inciHospdf[curAge+'_Ingen vaccination'] = curNone
    inciHospdf[curAge+'_Første vaccination'] = curOnce
    inciHospdf[curAge+'_Anden vaccination'] = curBoth 
    inciHospdf[curAge+'_Fuld vaccineeffekt'] = curFull
    
    # Add to smaller dataframes as well
    inciHospdf_none[curAge] = curNone
    inciHospdf_once[curAge] = curOnce
    inciHospdf_both[curAge] = curBoth 
    inciHospdf_full[curAge] = curFull
    

# df2_H
# curRow
inciHospdf
# cur

,Ugenummer,12-15_Ingen vaccination,12-15_Første vaccination,12-15_Anden vaccination,12-15_Fuld vaccineeffekt,16-19_Ingen vaccination,16-19_Første vaccination,16-19_Anden vaccination,16-19_Fuld vaccineeffekt,20-29_Ingen vaccination,...,70-79_Anden vaccination,70-79_Fuld vaccineeffekt,80+_Ingen vaccination,80+_Første vaccination,80+_Anden vaccination,80+_Fuld vaccineeffekt,12+_Ingen vaccination,12+_Første vaccination,12+_Anden vaccination,12+_Fuld vaccineeffekt
0,Uge 40,2.9,0.0,0.0,0.0,6.2,0.0,0.5,0.5,6.4,...,1.8,1.8,11.1,115.6,8.4,8.4,8.5,3.0,1.3,1.3
1,Uge 39,0.0,0.0,0.6,0.6,0.0,19.6,0.0,0.0,5.7,...,2.7,2.8,11.0,110.9,4.2,4.2,6.3,5.2,1.0,1.0
2,Uge 38,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.9,...,3.1,3.1,21.7,0.0,9.1,9.1,5.1,2.1,1.3,1.3
3,Uge 37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,2.0,2.0,0.0,0.0,6.8,6.8,5.6,4.3,1.1,1.1
4,Uge 36,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.4,...,2.6,2.6,31.8,0.0,7.6,7.6,12.8,6.1,1.5,1.5
5,Uge 35,4.7,0.0,0.0,0.0,8.2,0.0,0.0,0.0,10.5,...,2.2,2.2,94.4,179.9,6.8,6.4,18.8,6.2,1.2,1.2
6,Uge 34,3.2,1.2,0.0,0.0,13.1,0.0,0.5,0.5,10.1,...,4.0,4.0,62.0,0.0,6.4,6.4,18.1,3.5,1.5,1.7
7,Uge 33,1.9,0.0,0.0,0.0,14.9,0.0,0.5,0.0,18.7,...,2.9,2.9,30.7,78.2,4.1,4.2,19.3,2.7,1.6,1.7
8,Uge 32,3.3,0.8,0.0,0.0,7.1,3.5,0.6,0.6,12.7,...,1.8,1.8,40.4,0.0,3.4,3.4,14.3,3.8,1.1,1.1
9,Uge 31,1.3,0.0,0.0,0.0,4.5,2.1,0.0,0.0,10.0,...,1.1,1.1,0.0,0.0,4.9,4.9,7.9,1.5,1.2,1.2


In [17]:
allWeeks = df1['Ugenummer'].apply(lambda x: int(x[-2:]))

# Uge 36: 07-09-2021
weekOffset = allWeeks - 36
allDates = weekOffset.apply(lambda x:np.datetime64('2021-09-07') + np.timedelta64(x*7,'D'))

In [18]:
firstIndex = 4
lastIndex = 0
firstDate = allDates[firstIndex]
lastDate = allDates[lastIndex]

print(f'From {firstDate:%d. %B} until {lastDate:%d. %B}')

From 07. september until 05. oktober


In [19]:
sumInciFull = inciHospdf_full.iloc[lastIndex:firstIndex,1:].sum()
sumInciNone = inciHospdf_none.iloc[lastIndex:firstIndex,1:].sum()
ageGroups = inciHospdf_none.columns[1:]
fig,ax1 = plt.subplots()
# ax1.bar(ageGroups,sumInciNone)
ax1.plot(sumInciFull)
ax1.plot(sumInciNone)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# print(sumInciNone)
# print(sumInciFull)

# Idé: Rund op til 1 million, sammenlign antal personer
# Eller nej, faktisk bare rund af

# For hver 100.000 uvaccinerede danskere mellem 20 og 29, blev 21 personer indlagt i perioden.
# For hver 100.000 vaccinerede  danskere mellem 20 og 29, blev  2 personer indlagt i perioden.

roundNone = sumInciNone.round().astype(int)
roundFull = sumInciFull.round().astype(int)

clrNone = 'xkcd:light red'
clrFull = 'xkcd:light blue'

curAge = '20-29'
curAge = '70-79'
curAge = '80+'

def makePlot(curAge,saveTheFigure=False):
    curNone = roundNone[curAge]
    curFull = roundFull[curAge]

    fig,ax1 = plt.subplots(figsize=(12,12))

    x0n = 0
    y0n = 1
    x0f = 6
    y0f = 1
    hei = 0.7
    wid = 0.7

    curX = 0
    curY = 0
    
    maxX = 5
    if (curNone > 25):
        maxX = 10
        x0f = 11
    
    for k in range(0,curNone):
        if (curX >= maxX):
            curX = curX - maxX
            curY = curY + 1
            
        curX = curX + 1
        ax1.fill(x0n+curX+np.array([0,0,wid,wid]),y0n+curY+np.array([0,hei,hei,0]),color=clrNone)
        # ax1.plot(curX,curY,'o',color=clrNone)

    yTop = curY
        
    curX = 0
    curY = 0
    for k in range(0,curFull):
        if (curX >= maxX):
            curX = curX - maxX
            curY = curY + 1
            
        curX = curX + 1 
        ax1.fill(curX+x0f+np.array([0,0,wid,wid]),curY+y0f+np.array([0,hei,hei,0]),color=clrFull)
        # ax1.plot(curX,curY,'o',color=clrNone)

    # labelTop = f'Fra {firstDate:%d. %B} til {lastDate:%d. %B} blev:'
    # ageLabel = f'mellem {curAge[0:2]} og {curAge[-2:]}'
    # labelN = f'{curNone} personer indlagt \nfor hver 100.000 uvaccinerede\ndanskere {ageLabel}'
    # labelF = f'{curFull} personer indlagt \nfor hver 100.000 vaccinerede\ndanskere {ageLabel}'
    # # labelF = f'For hver 100.000 vaccinerede\ndanskere mellem 20 og 29,\nblev {curFull} personer.'
    # ax1.text(x0n+1 + maxX +0.5 ,yTop + 4,labelTop,ha='center',va='center',size=20)
    # ax1.text(x0n+1 + maxX/2,yTop + 2.5,labelN,ha='center',va='center',size=14)
    # ax1.text(x0f+1 + maxX/2,yTop + 2.5,labelF,ha='center',va='center',size=14)


    labelTop = f'Blandt {curAge[0:2]} til {curAge[-2:]} årige\nfra {firstDate:%d. %B} til {lastDate:%d. %B} blev:'
    if (curAge == '80+'):
        labelTop = f'Blandt personer over 80 år \nfra {firstDate:%d. %B} til {lastDate:%d. %B} blev:'
        
    if (curAge == '12+'):
        labelTop = f'Blandt alle over 12 år \nfra {firstDate:%d. %B} til {lastDate:%d. %B} blev:'
    labelN = f'{curNone} personer indlagt \nfor hver 100.000 uvaccinerede'
    labelF = f'{curFull} personer indlagt \nfor hver 100.000 vaccinerede'
    if (curFull == 1):
        labelF = f'1 person indlagt \nfor hver 100.000 vaccinerede'
    # labelF = f'For hver 100.000 vaccinerede\ndanskere mellem 20 og 29,\nblev {curFull} personer.'
    ax1.text(x0n+1 + maxX +0.5 ,yTop + 4,labelTop,ha='center',va='center',size=18)
    ax1.text(x0n+1 + maxX/2,yTop + 2.5,labelN,ha='center',va='center',size=14)
    ax1.text(x0f+1 + maxX/2,yTop + 2.5,labelF,ha='center',va='center',size=14)

    # Set limits
    xWidth = maxX*2+2
    if ((yTop+5) > xWidth):
        xWidth = yTop + 5
    print(yTop)
    ax1.set_xlim([0,xWidth])
    ax1.set_ylim([0,xWidth])
    # plt.axis('equal')

    # Hide axes and labels
    ax1.spines['top'].set_visible(False)
    ax1.spines['bottom'].set_visible(False)
    ax1.spines['left'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.set_xticks([])
    ax1.set_yticks([])

    # ax1.set_xticklabels('')
    # plt.axis('equal')
    # plt.tight_layout() 
    
    if saveTheFigure:
        plt.savefig('figs/gennembrud/Visual_'+curAge)


In [21]:

for curAge in inciHospdf_none.columns[1:]:
    makePlot(curAge,saveTheFigure=True)
# makePlot('70-79',saveTheFigure=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2


In [23]:
# Dødsfald, 80+

roundNone = sumInciNone.round().astype(int)
roundFull = sumInciFull.round().astype(int)

clrNone = 'xkcd:light red'
clrFull = 'xkcd:light blue'

curAge = '20-29'
curAge = '70-79'
curAge = '80+'

curNone = 109
curFull = 9

fig,ax1 = plt.subplots(figsize=(12,12))

x0n = 0
y0n = 1
x0f = 6
y0f = 1
hei = 0.7
wid = 0.7

curX = 0
curY = 0

maxX = 5
if (curNone > 25):
    maxX = 10
    x0f = 11

for k in range(0,curNone):
    if (curX >= maxX):
        curX = curX - maxX
        curY = curY + 1
        
    curX = curX + 1
    ax1.fill(x0n+curX+np.array([0,0,wid,wid]),y0n+curY+np.array([0,hei,hei,0]),color=clrNone)
    # ax1.plot(curX,curY,'o',color=clrNone)

yTop = curY
    
curX = 0
curY = 0
for k in range(0,curFull):
    if (curX >= maxX):
        curX = curX - maxX
        curY = curY + 1
        
    curX = curX + 1 
    ax1.fill(curX+x0f+np.array([0,0,wid,wid]),curY+y0f+np.array([0,hei,hei,0]),color=clrFull)


# labelTop = f'For personer over 80 år \nfra {firstDate:%d. %B} til {lastDate:%d. %B}:'
labelTop = f'For personer over 80 år \nmellem uge 34 og 37:'


labelN = f'{curNone} dødsfald \nfor hver 100.000 uvaccinerede'
labelF = f'{curFull} dødsfald \nfor hver 100.000 vaccinerede'
if (curFull == 1):
    labelF = f'1 person indlagt \nfor hver 100.000 vaccinerede'
# labelF = f'For hver 100.000 vaccinerede\ndanskere mellem 20 og 29,\nblev {curFull} personer.'
ax1.text(x0n+1 + maxX +0.5 ,yTop + 4,labelTop,ha='center',va='center',size=18)
ax1.text(x0n+1 + maxX/2,yTop + 2.5,labelN,ha='center',va='center',size=14)
ax1.text(x0f+1 + maxX/2,yTop + 2.5,labelF,ha='center',va='center',size=14)

# Set limits
xWidth = maxX*2+2
if ((yTop+5) > xWidth):
    xWidth = yTop + 5
print(yTop)
ax1.set_xlim([0,xWidth])
ax1.set_ylim([0,xWidth])
# plt.axis('equal')

# Hide axes and labels
ax1.spines['top'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_xticks([])
ax1.set_yticks([])

# ax1.set_xticklabels('')
# plt.axis('equal')
# plt.tight_layout() 

# if saveTheFigure:
plt.savefig('figs/gennembrud/Visual_Dead_80+')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

10
